## Sales Prediction & Modeling Using XGBoost

In [54]:
import os.path
import pandas as pd
import numpy as np
import xgboost as xgb 

base_path = "./data/"

### Import Sales Data

In [55]:
navigation = pd.read_csv(os.path.join(base_path, 'navigation.csv'))
sales = pd.read_csv(os.path.join(base_path, 'sales.csv'))
train = pd.read_csv(os.path.join(base_path, 'train.csv'))
test = pd.read_csv(os.path.join(base_path, 'test.csv'))

In [56]:
train.head()

,ID,month,sku_hash,product_type,product_gender,macro_function,function,sub_function,model,aesthetic_sub_line,macro_material,color,fr_FR_price,en_US_description,target
0,3d8a4ae769b526187c36901f204691a663333fa4_1,1,3d8a4ae769b526187c36901f204691a663333fa4,Leather Goods,Women,LG ACCESSORIES,SMALL LEATHER GOODS,COMPACT WALLETS,PF.VICTORINE,DAMIER AZUR,CANVAS,AZUR,370.0,"Fashioned from summery Damier Azur canvas, thi...",1366.0
1,c05a54f7067be054ec4b27d0d6081353ef7d9df6_1,1,c05a54f7067be054ec4b27d0d6081353ef7d9df6,Accessories,Women,FANCY ACCESSORIES,BAG CHARMS,BAG CHARMS,BAG CHARM PETITE MALLE,SANS LIGNE ESTHETIQUE,CANVAS,NaN,295.0,This collectible piece features our iconic Pet...,526.0
2,7cb4d3626bd48a9b523d8693266219c34aeccde8_1,1,7cb4d3626bd48a9b523d8693266219c34aeccde8,Leather Goods,Women,CITY BAGS,DAILY BAGS,SHOULDER BAGS,GIROLATA,MAHINA,LEATHER,GALET,2660.0,"The perfect summer companion, this effortlessl...",503.0
3,8e5967f90ff9fb93aa2840fceecc537b2e8e6ff3_1,1,8e5967f90ff9fb93aa2840fceecc537b2e8e6ff3,Leather Goods,Women,CITY BAGS,DAILY BAGS,SHOULDER BAGS,GIROLATA,MAHINA,LEATHER,MAGNOLIA,2660.0,"The perfect summer companion, this effortlessl...",276.0
4,6913a128945e0efeafc52101dcdeaa610eaa4430_1,1,6913a128945e0efeafc52101dcdeaa610eaa4430,Accessories,Men,FASHION JEWELRY,NECKLACES,NECKLACES,COLL.CHARMS FOR GENTLEMEN,SANS LIGNE ESTHETIQUE,NON PRECIOUS METAL,NaN,325.0,This distinctive charm necklace reinterprets d...,206.0


In [57]:
sales.head()

,Date,day_transaction_date,Month_transaction,type,zone_number,country_number,name,sku_hash,sales_quantity,currency_rate_USD,...,NetSentiment_6_day_before,PositiveSentiment_6_day_before,NegativeSentiment_6_day_before,Impressions_6_day_before,TotalBuzzPost_7_day_before,TotalBuzz_7_day_before,NetSentiment_7_day_before,PositiveSentiment_7_day_before,NegativeSentiment_7_day_before,Impressions_7_day_before
0,Day_7,Thursday,4,Type_2,4,43,Name_1,84d4e90b67c2b9902cff522be4eb121f637c17b5,5,1.225636,...,77.525599,33634.0,4258.0,643226023.0,68960.0,107970.0,70.473656,32032.0,5548.0,631477319.0
1,Day_6,Wednesday,4,Type_1,4,20,Name_1,84d4e90b67c2b9902cff522be4eb121f637c17b5,5,1.228951,...,70.473656,32032.0,5548.0,631477319.0,65312.0,102920.0,71.788703,34277.0,5629.0,730132649.0
2,Day_1,Monday,1,Type_2,2,65,Name_1,70f3dd559074ecfc9d69ff64480bc8acae977e3a,5,1.201496,...,46.158532,27594.0,10165.0,359205211.0,45746.0,64881.0,73.083627,21349.0,3320.0,268925766.0
3,Day_7,Sunday,1,Type_1,3,49,Name_1,70f3dd559074ecfc9d69ff64480bc8acae977e3a,5,1.204091,...,72.438611,16292.0,2604.0,307167067.0,42796.0,61653.0,68.744442,20872.0,3866.0,323820589.0
4,Day_5,Tuesday,2,Type_1,4,20,Name_1,1f8b7a2b42d4cdf5eef70f75fb86a34d514d2747,5,1.238391,...,72.776593,28344.0,4466.0,440849942.0,60653.0,90848.0,75.338753,26527.0,3731.0,393463263.0


### Product Target Average

In [58]:
product_descriptor = ['product_type', 'product_gender', 'macro_function', 
                      'function', 'sub_function', 'model', 'aesthetic_sub_line', 'macro_material',
                      'month']
product_target_sum = train.groupby(product_descriptor)['target'].sum().reset_index(name = 'sum_target')
product_target_count = train.groupby(product_descriptor)['target'].count().reset_index(name = 'count_target')
product_target_stats = pd.merge(product_target_sum,product_target_count,on=product_descriptor)

train = train.merge(product_target_stats,on=product_descriptor,how='left')
test = test.merge(product_target_stats,on=product_descriptor,how='left')

train['mean_target'] = (train['sum_target'] - train['target'])/(train['count_target']-1)
test['mean_target'] = test['sum_target']/test['count_target']


In [59]:
train.head()

,ID,month,sku_hash,product_type,product_gender,macro_function,function,sub_function,model,aesthetic_sub_line,macro_material,color,fr_FR_price,en_US_description,target,sum_target,count_target,mean_target
0,3d8a4ae769b526187c36901f204691a663333fa4_1,1,3d8a4ae769b526187c36901f204691a663333fa4,Leather Goods,Women,LG ACCESSORIES,SMALL LEATHER GOODS,COMPACT WALLETS,PF.VICTORINE,DAMIER AZUR,CANVAS,AZUR,370.0,"Fashioned from summery Damier Azur canvas, thi...",1366.0,1837.0,2,471.000000
1,c05a54f7067be054ec4b27d0d6081353ef7d9df6_1,1,c05a54f7067be054ec4b27d0d6081353ef7d9df6,Accessories,Women,FANCY ACCESSORIES,BAG CHARMS,BAG CHARMS,BAG CHARM PETITE MALLE,SANS LIGNE ESTHETIQUE,CANVAS,NaN,295.0,This collectible piece features our iconic Pet...,526.0,526.0,1,NaN
2,7cb4d3626bd48a9b523d8693266219c34aeccde8_1,1,7cb4d3626bd48a9b523d8693266219c34aeccde8,Leather Goods,Women,CITY BAGS,DAILY BAGS,SHOULDER BAGS,GIROLATA,MAHINA,LEATHER,GALET,2660.0,"The perfect summer companion, this effortlessl...",503.0,1395.0,4,297.333333
3,8e5967f90ff9fb93aa2840fceecc537b2e8e6ff3_1,1,8e5967f90ff9fb93aa2840fceecc537b2e8e6ff3,Leather Goods,Women,CITY BAGS,DAILY BAGS,SHOULDER BAGS,GIROLATA,MAHINA,LEATHER,MAGNOLIA,2660.0,"The perfect summer companion, this effortlessl...",276.0,1395.0,4,373.000000
4,6913a128945e0efeafc52101dcdeaa610eaa4430_1,1,6913a128945e0efeafc52101dcdeaa610eaa4430,Accessories,Men,FASHION JEWELRY,NECKLACES,NECKLACES,COLL.CHARMS FOR GENTLEMEN,SANS LIGNE ESTHETIQUE,NON PRECIOUS METAL,NaN,325.0,This distinctive charm necklace reinterprets d...,206.0,206.0,1,NaN


In [60]:
train.drop(['count_target','sum_target'],axis=1) 
test.drop(['count_target','sum_target'],axis=1)

,ID,month,sku_hash,product_type,product_gender,macro_function,function,sub_function,model,aesthetic_sub_line,macro_material,color,fr_FR_price,en_US_description,mean_target
0,ed4c7471eac7e8c6e6718364c2b6e75462eeb47c_1,1,ed4c7471eac7e8c6e6718364c2b6e75462eeb47c,Accessories,Women,TEXTILES,SCARVES,WINTER SCARVES,ECHARPE REYKJAVIK,SANS LIGNE ESTHETIQUE,TEXTILE,NaN,650.0,"In pure, soft cashmere, the Reykjavik scarf is...",782.0
1,26b2c4f6281482cccf1e748ef388f1649ecf1c8b_1,1,26b2c4f6281482cccf1e748ef388f1649ecf1c8b,Leather Goods,Women,CITY BAGS,DAILY BAGS,HANDBAGS,VERY ZIPPED TOTE,VERY,LEATHER,KAKI FANGO COQUILLE,2550.0,"For Fall-Winter 2018, the Very Zipped Tote han...",179.0
2,00aad36ebc04aaa761df219c1b48d1e6162d28bb_1,1,00aad36ebc04aaa761df219c1b48d1e6162d28bb,Leather Goods,Women,CITY BAGS,DAILY BAGS,CROSS SHOULDER BAGS,HINA,MAHINA,LEATHER,NOIR,2460.0,The Hina is two bags in one: gather in the sid...,NaN
3,8281636e7d162c92c1a3c1f9e40afb05d39a02f9_1,1,8281636e7d162c92c1a3c1f9e40afb05d39a02f9,Accessories,Men,TEXTILES,STOLES,WINTER STOLES,ETOLE MONOGRAM NIGHTFALL,SANS LIGNE ESTHETIQUE,TEXTILE,NaN,455.0,With its subtle tone-on-tone Monogram pattern ...,NaN
4,b76f8505700409a198fe69fb33e5d6cc01166c9b_1,1,b76f8505700409a198fe69fb33e5d6cc01166c9b,Accessories,Men,TIES,TIES,OTHER TIES,CRAVATE MONOGRAM ILLUSION,SANS LIGNE ESTHETIQUE,TEXTILE,NaN,150.0,Suitable both for business and special occasio...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5161,ee17648d76504b8f090e1d8e4df64486f5b91207_3,3,ee17648d76504b8f090e1d8e4df64486f5b91207,Leather Goods,Men,CITY BAGS,CLUTCHES,CLUTCHES BAGS,POCHETTE KASAI,TAIGA,LEATHER,BLEU MARINE,970.0,"In supple Taiga leather, the discreet Clutch K...",187.0
5162,a774300439ee82752493490273c0b2b20a547ae8_3,3,a774300439ee82752493490273c0b2b20a547ae8,Accessories,Women,TEXTILES,SET HAT & GLOVES,GLOVES SETS,GANTS MORNINGRAM,SANS LIGNE ESTHETIQUE,TEXTILE,NaN,295.0,"Stylish and cozy in cold weather, the Morningr...",NaN
5163,312eab28efd44e9006a3ad9d598f144bbb4af37e_3,3,312eab28efd44e9006a3ad9d598f144bbb4af37e,Accessories,Women,EYEWEAR,SUNGLASSES,METAL SUNGLASSES,MIDNIGHT IN PARIS,SANS LIGNE ESTHETIQUE,NON PRECIOUS METAL,NaN,480.0,An openwork metal frame gives these cat-eye su...,NaN
5164,45a654a8d97c04344dbebc023cb89c55bd6bec77_3,3,45a654a8d97c04344dbebc023cb89c55bd6bec77,Accessories,Women,FASHION JEWELRY,BRACELETS,FANCY BRACELETS,JONC STUDDY,SANS LIGNE ESTHETIQUE,NON PRECIOUS METAL,NaN,375.0,Inspired by the iconic studs from Louis Vuitto...,NaN


In [61]:
train.head()

,ID,month,sku_hash,product_type,product_gender,macro_function,function,sub_function,model,aesthetic_sub_line,macro_material,color,fr_FR_price,en_US_description,target,sum_target,count_target,mean_target
0,3d8a4ae769b526187c36901f204691a663333fa4_1,1,3d8a4ae769b526187c36901f204691a663333fa4,Leather Goods,Women,LG ACCESSORIES,SMALL LEATHER GOODS,COMPACT WALLETS,PF.VICTORINE,DAMIER AZUR,CANVAS,AZUR,370.0,"Fashioned from summery Damier Azur canvas, thi...",1366.0,1837.0,2,471.000000
1,c05a54f7067be054ec4b27d0d6081353ef7d9df6_1,1,c05a54f7067be054ec4b27d0d6081353ef7d9df6,Accessories,Women,FANCY ACCESSORIES,BAG CHARMS,BAG CHARMS,BAG CHARM PETITE MALLE,SANS LIGNE ESTHETIQUE,CANVAS,NaN,295.0,This collectible piece features our iconic Pet...,526.0,526.0,1,NaN
2,7cb4d3626bd48a9b523d8693266219c34aeccde8_1,1,7cb4d3626bd48a9b523d8693266219c34aeccde8,Leather Goods,Women,CITY BAGS,DAILY BAGS,SHOULDER BAGS,GIROLATA,MAHINA,LEATHER,GALET,2660.0,"The perfect summer companion, this effortlessl...",503.0,1395.0,4,297.333333
3,8e5967f90ff9fb93aa2840fceecc537b2e8e6ff3_1,1,8e5967f90ff9fb93aa2840fceecc537b2e8e6ff3,Leather Goods,Women,CITY BAGS,DAILY BAGS,SHOULDER BAGS,GIROLATA,MAHINA,LEATHER,MAGNOLIA,2660.0,"The perfect summer companion, this effortlessl...",276.0,1395.0,4,373.000000
4,6913a128945e0efeafc52101dcdeaa610eaa4430_1,1,6913a128945e0efeafc52101dcdeaa610eaa4430,Accessories,Men,FASHION JEWELRY,NECKLACES,NECKLACES,COLL.CHARMS FOR GENTLEMEN,SANS LIGNE ESTHETIQUE,NON PRECIOUS METAL,NaN,325.0,This distinctive charm necklace reinterprets d...,206.0,206.0,1,NaN


In [62]:
train.drop(['count_target','sum_target'],axis=1,inplace =True) 
test.drop(['count_target','sum_target'],axis=1,inplace =True)

In [63]:
train.head()

,ID,month,sku_hash,product_type,product_gender,macro_function,function,sub_function,model,aesthetic_sub_line,macro_material,color,fr_FR_price,en_US_description,target,mean_target
0,3d8a4ae769b526187c36901f204691a663333fa4_1,1,3d8a4ae769b526187c36901f204691a663333fa4,Leather Goods,Women,LG ACCESSORIES,SMALL LEATHER GOODS,COMPACT WALLETS,PF.VICTORINE,DAMIER AZUR,CANVAS,AZUR,370.0,"Fashioned from summery Damier Azur canvas, thi...",1366.0,471.000000
1,c05a54f7067be054ec4b27d0d6081353ef7d9df6_1,1,c05a54f7067be054ec4b27d0d6081353ef7d9df6,Accessories,Women,FANCY ACCESSORIES,BAG CHARMS,BAG CHARMS,BAG CHARM PETITE MALLE,SANS LIGNE ESTHETIQUE,CANVAS,NaN,295.0,This collectible piece features our iconic Pet...,526.0,NaN
2,7cb4d3626bd48a9b523d8693266219c34aeccde8_1,1,7cb4d3626bd48a9b523d8693266219c34aeccde8,Leather Goods,Women,CITY BAGS,DAILY BAGS,SHOULDER BAGS,GIROLATA,MAHINA,LEATHER,GALET,2660.0,"The perfect summer companion, this effortlessl...",503.0,297.333333
3,8e5967f90ff9fb93aa2840fceecc537b2e8e6ff3_1,1,8e5967f90ff9fb93aa2840fceecc537b2e8e6ff3,Leather Goods,Women,CITY BAGS,DAILY BAGS,SHOULDER BAGS,GIROLATA,MAHINA,LEATHER,MAGNOLIA,2660.0,"The perfect summer companion, this effortlessl...",276.0,373.000000
4,6913a128945e0efeafc52101dcdeaa610eaa4430_1,1,6913a128945e0efeafc52101dcdeaa610eaa4430,Accessories,Men,FASHION JEWELRY,NECKLACES,NECKLACES,COLL.CHARMS FOR GENTLEMEN,SANS LIGNE ESTHETIQUE,NON PRECIOUS METAL,NaN,325.0,This distinctive charm necklace reinterprets d...,206.0,NaN


### Count Product by Hash Code

In [64]:
count_vec_cols = ['macro_function', 'function', 'sub_function', 'model',
                  'aesthetic_sub_line', 'macro_material', 'color']

for col in count_vec_cols:
    tmp = pd.DataFrame({'sku_hash':pd.concat([train['sku_hash'],test['sku_hash']]),col:pd.concat([train[col],test[col]])})
    tmp = pd.DataFrame(tmp.groupby(col)['sku_hash'].count()).reset_index()
    tmp.columns = [col,col+'_count']
    
    train = train.merge(tmp,on=col,how='left')
    test = test.merge(tmp,on=col,how='left')

In [65]:
train.head()

,ID,month,sku_hash,product_type,product_gender,macro_function,function,sub_function,model,aesthetic_sub_line,...,en_US_description,target,mean_target,macro_function_count,function_count,sub_function_count,model_count,aesthetic_sub_line_count,macro_material_count,color_count
0,3d8a4ae769b526187c36901f204691a663333fa4_1,1,3d8a4ae769b526187c36901f204691a663333fa4,Leather Goods,Women,LG ACCESSORIES,SMALL LEATHER GOODS,COMPACT WALLETS,PF.VICTORINE,DAMIER AZUR,...,"Fashioned from summery Damier Azur canvas, thi...",1366.0,471.000000,2292,2094,447,90,126,3213,120.0
1,c05a54f7067be054ec4b27d0d6081353ef7d9df6_1,1,c05a54f7067be054ec4b27d0d6081353ef7d9df6,Accessories,Women,FANCY ACCESSORIES,BAG CHARMS,BAG CHARMS,BAG CHARM PETITE MALLE,SANS LIGNE ESTHETIQUE,...,This collectible piece features our iconic Pet...,526.0,NaN,546,126,102,3,3477,3213,NaN
2,7cb4d3626bd48a9b523d8693266219c34aeccde8_1,1,7cb4d3626bd48a9b523d8693266219c34aeccde8,Leather Goods,Women,CITY BAGS,DAILY BAGS,SHOULDER BAGS,GIROLATA,MAHINA,...,"The perfect summer companion, this effortlessl...",503.0,297.333333,3039,2784,699,15,90,4089,39.0
3,8e5967f90ff9fb93aa2840fceecc537b2e8e6ff3_1,1,8e5967f90ff9fb93aa2840fceecc537b2e8e6ff3,Leather Goods,Women,CITY BAGS,DAILY BAGS,SHOULDER BAGS,GIROLATA,MAHINA,...,"The perfect summer companion, this effortlessl...",276.0,373.000000,3039,2784,699,15,90,4089,60.0
4,6913a128945e0efeafc52101dcdeaa610eaa4430_1,1,6913a128945e0efeafc52101dcdeaa610eaa4430,Accessories,Men,FASHION JEWELRY,NECKLACES,NECKLACES,COLL.CHARMS FOR GENTLEMEN,SANS LIGNE ESTHETIQUE,...,This distinctive charm necklace reinterprets d...,206.0,NaN,843,171,162,3,3477,1023,NaN


### Source View Statistics

In [66]:
traffic_source_views = navigation.groupby(['sku_hash','traffic_source'])['page_views'].sum().reset_index()

In [67]:
traffic_source_views[:5]

,sku_hash,traffic_source,page_views
0,000cb631113e2f54ca5512139a6592e9584957aa,Source 1,1748
1,000cb631113e2f54ca5512139a6592e9584957aa,Source 2,40
2,000cb631113e2f54ca5512139a6592e9584957aa,Source 5,30
3,000cb631113e2f54ca5512139a6592e9584957aa,Source 6,48
4,0020d561eab8b88ab55dfde84a2f12b865e5e0b4,Source 1,16300


In [68]:
traffic_source_views = traffic_source_views.pivot(index = 'sku_hash',columns = 'traffic_source',values = 'page_views').reset_index()

In [69]:
traffic_source_views.columns = ['sku_hash', 
                                'page_views_nav1', 'page_views_nav2', 'page_views_nav3', 
                                'page_views_nav4', 'page_views_nav5', 'page_views_nav6']

In [70]:
traffic_source_views.head()

,sku_hash,page_views_nav1,page_views_nav2,page_views_nav3,page_views_nav4,page_views_nav5,page_views_nav6
0,000cb631113e2f54ca5512139a6592e9584957aa,1748.0,40.0,NaN,NaN,30.0,48.0
1,0020d561eab8b88ab55dfde84a2f12b865e5e0b4,16300.0,286.0,NaN,25.0,93.0,25.0
2,0026e7a0fcfe5999a44b70b1acaff00fc1ad3ac2,147.0,5.0,NaN,NaN,5.0,NaN
3,00287bbb94c12066df6491dccd744ee87ff01a90,3473.0,15.0,NaN,25.0,NaN,NaN
4,003f8a76cb823eb7c58b6d052c57a8933f9275fd,47468.0,251.0,NaN,55.0,NaN,10.0


### Product Type Sales

In [71]:
type_sales = sales.groupby(['sku_hash','type'])['sales_quantity'].sum().reset_index()
type_sales = type_sales.pivot(index = 'sku_hash',columns = 'type',values = 'sales_quantity').reset_index()
type_sales.columns = ['sku_hash', 'sales_quantity_type1', 'sales_quantity_type2']

In [72]:
type_sales.head()

,sku_hash,sales_quantity_type1,sales_quantity_type2
0,000cb631113e2f54ca5512139a6592e9584957aa,104.0,10.0
1,0020d561eab8b88ab55dfde84a2f12b865e5e0b4,325.0,40.0
2,0026e7a0fcfe5999a44b70b1acaff00fc1ad3ac2,85.0,25.0
3,00287bbb94c12066df6491dccd744ee87ff01a90,35.0,10.0
4,003f8a76cb823eb7c58b6d052c57a8933f9275fd,253.0,145.0


### Regional Sales Statistics

In [73]:
zone_sales = sales.groupby(['sku_hash','zone_number'])['sales_quantity'].sum().reset_index()

zone_sales = zone_sales.pivot(index = 'sku_hash',columns = 'zone_number',values = 'sales_quantity').reset_index()
zone_sales.columns = ['sku_hash', 
                      'sales_quantity_zone1', 'sales_quantity_zone2', 'sales_quantity_zone3', 
                      'sales_quantity_zone4', 'sales_quantity_zone5']

In [74]:
zone_sales.head()

,sku_hash,sales_quantity_zone1,sales_quantity_zone2,sales_quantity_zone3,sales_quantity_zone4,sales_quantity_zone5
0,000cb631113e2f54ca5512139a6592e9584957aa,20.0,5.0,15.0,50.0,24.0
1,0020d561eab8b88ab55dfde84a2f12b865e5e0b4,NaN,NaN,82.0,20.0,263.0
2,0026e7a0fcfe5999a44b70b1acaff00fc1ad3ac2,55.0,35.0,10.0,NaN,10.0
3,00287bbb94c12066df6491dccd744ee87ff01a90,10.0,NaN,5.0,30.0,NaN
4,003f8a76cb823eb7c58b6d052c57a8933f9275fd,45.0,148.0,50.0,140.0,15.0


### Product View Statistics

In [75]:
navigation_stats = navigation.groupby(['sku_hash'])['page_views'].sum().reset_index(name='page_views')
sales_stats = sales.groupby(['sku_hash'])['sales_quantity','TotalBuzzPost', 'TotalBuzz','NetSentiment', 'PositiveSentiment', 'NegativeSentiment', 'Impressions'].sum().reset_index()

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [76]:
navigation_stats.head()

,sku_hash,page_views
0,000cb631113e2f54ca5512139a6592e9584957aa,1866
1,0020d561eab8b88ab55dfde84a2f12b865e5e0b4,16729
2,0026e7a0fcfe5999a44b70b1acaff00fc1ad3ac2,157
3,00287bbb94c12066df6491dccd744ee87ff01a90,3513
4,003f8a76cb823eb7c58b6d052c57a8933f9275fd,47784


In [77]:
sales_stats.head()

,sku_hash,sales_quantity,TotalBuzzPost,TotalBuzz,NetSentiment,PositiveSentiment,NegativeSentiment,Impressions
0,000cb631113e2f54ca5512139a6592e9584957aa,114,1308458.0,1791611.0,1471.884478,469288.0,67668.0,8.729030e+09
1,0020d561eab8b88ab55dfde84a2f12b865e5e0b4,365,497215.0,803633.0,851.430118,228523.0,38871.0,4.159187e+09
2,0026e7a0fcfe5999a44b70b1acaff00fc1ad3ac2,110,1206926.0,1718843.0,1380.767295,524646.0,100366.0,1.121427e+10
3,00287bbb94c12066df6491dccd744ee87ff01a90,45,475098.0,680543.0,581.175056,211602.0,33527.0,3.253251e+09
4,003f8a76cb823eb7c58b6d052c57a8933f9275fd,398,3282466.0,4795323.0,3742.253384,1401916.0,307473.0,2.389471e+10


### Divide the training set

In [78]:
train['idx'] = pd.Categorical(train.sku_hash).codes
train['idx'] = train['idx'] % 5
train.head()

,ID,month,sku_hash,product_type,product_gender,macro_function,function,sub_function,model,aesthetic_sub_line,...,target,mean_target,macro_function_count,function_count,sub_function_count,model_count,aesthetic_sub_line_count,macro_material_count,color_count,idx
0,3d8a4ae769b526187c36901f204691a663333fa4_1,1,3d8a4ae769b526187c36901f204691a663333fa4,Leather Goods,Women,LG ACCESSORIES,SMALL LEATHER GOODS,COMPACT WALLETS,PF.VICTORINE,DAMIER AZUR,...,1366.0,471.000000,2292,2094,447,90,126,3213,120.0,1
1,c05a54f7067be054ec4b27d0d6081353ef7d9df6_1,1,c05a54f7067be054ec4b27d0d6081353ef7d9df6,Accessories,Women,FANCY ACCESSORIES,BAG CHARMS,BAG CHARMS,BAG CHARM PETITE MALLE,SANS LIGNE ESTHETIQUE,...,526.0,NaN,546,126,102,3,3477,3213,NaN,1
2,7cb4d3626bd48a9b523d8693266219c34aeccde8_1,1,7cb4d3626bd48a9b523d8693266219c34aeccde8,Leather Goods,Women,CITY BAGS,DAILY BAGS,SHOULDER BAGS,GIROLATA,MAHINA,...,503.0,297.333333,3039,2784,699,15,90,4089,39.0,4
3,8e5967f90ff9fb93aa2840fceecc537b2e8e6ff3_1,1,8e5967f90ff9fb93aa2840fceecc537b2e8e6ff3,Leather Goods,Women,CITY BAGS,DAILY BAGS,SHOULDER BAGS,GIROLATA,MAHINA,...,276.0,373.000000,3039,2784,699,15,90,4089,60.0,2
4,6913a128945e0efeafc52101dcdeaa610eaa4430_1,1,6913a128945e0efeafc52101dcdeaa610eaa4430,Accessories,Men,FASHION JEWELRY,NECKLACES,NECKLACES,COLL.CHARMS FOR GENTLEMEN,SANS LIGNE ESTHETIQUE,...,206.0,NaN,843,171,162,3,3477,1023,NaN,0


### Merge Training Dataset

In [79]:
X = train.copy()
X = X.merge(navigation_stats, on = 'sku_hash', how = 'left')
X = X.merge(sales_stats, on = 'sku_hash', how = 'left')
X = X.merge(traffic_source_views, on = 'sku_hash', how = 'left')
X = X.merge(type_sales, on = 'sku_hash', how = 'left')
X = X.merge(zone_sales, on = 'sku_hash', how = 'left')
X.head()

,ID,month,sku_hash,product_type,product_gender,macro_function,function,sub_function,model,aesthetic_sub_line,...,page_views_nav4,page_views_nav5,page_views_nav6,sales_quantity_type1,sales_quantity_type2,sales_quantity_zone1,sales_quantity_zone2,sales_quantity_zone3,sales_quantity_zone4,sales_quantity_zone5
0,3d8a4ae769b526187c36901f204691a663333fa4_1,1,3d8a4ae769b526187c36901f204691a663333fa4,Leather Goods,Women,LG ACCESSORIES,SMALL LEATHER GOODS,COMPACT WALLETS,PF.VICTORINE,DAMIER AZUR,...,20.0,25.0,29.0,946.0,554.0,506.0,238.0,75.0,544.0,137.0
1,c05a54f7067be054ec4b27d0d6081353ef7d9df6_1,1,c05a54f7067be054ec4b27d0d6081353ef7d9df6,Accessories,Women,FANCY ACCESSORIES,BAG CHARMS,BAG CHARMS,BAG CHARM PETITE MALLE,SANS LIGNE ESTHETIQUE,...,NaN,39.0,NaN,338.0,60.0,84.0,55.0,25.0,155.0,79.0
2,7cb4d3626bd48a9b523d8693266219c34aeccde8_1,1,7cb4d3626bd48a9b523d8693266219c34aeccde8,Leather Goods,Women,CITY BAGS,DAILY BAGS,SHOULDER BAGS,GIROLATA,MAHINA,...,10.0,10.0,5.0,429.0,145.0,190.0,55.0,35.0,195.0,99.0
3,8e5967f90ff9fb93aa2840fceecc537b2e8e6ff3_1,1,8e5967f90ff9fb93aa2840fceecc537b2e8e6ff3,Leather Goods,Women,CITY BAGS,DAILY BAGS,SHOULDER BAGS,GIROLATA,MAHINA,...,5.0,10.0,5.0,228.0,55.0,108.0,55.0,20.0,20.0,80.0
4,6913a128945e0efeafc52101dcdeaa610eaa4430_1,1,6913a128945e0efeafc52101dcdeaa610eaa4430,Accessories,Men,FASHION JEWELRY,NECKLACES,NECKLACES,COLL.CHARMS FOR GENTLEMEN,SANS LIGNE ESTHETIQUE,...,NaN,5.0,NaN,110.0,25.0,35.0,5.0,20.0,30.0,45.0


In [80]:
X = train.copy()
X = X.merge(navigation_stats, on = 'sku_hash', how = 'left')
X = X.merge(sales_stats, on = 'sku_hash', how = 'left')
X = X.merge(traffic_source_views, on = 'sku_hash', how = 'left')
X = X.merge(type_sales, on = 'sku_hash', how = 'left')
X = X.merge(zone_sales, on = 'sku_hash', how = 'left')

X.loc[X.product_type=='Accessories','product_type'] = '0'
X.loc[X.product_type=='Leather Goods','product_type'] = '1'
X.product_type = X.product_type.astype(int)

X.loc[X.product_gender=='Women','product_gender'] = '-1'
X.loc[X.product_gender=='Unisex','product_gender'] = '0'
X.loc[X.product_gender=='Men','product_gender'] = '1'
X.product_gender = X.product_gender.astype(int)

X['y'] = np.log(X['target']+1)

In [81]:
X.head()

,ID,month,sku_hash,product_type,product_gender,macro_function,function,sub_function,model,aesthetic_sub_line,...,page_views_nav5,page_views_nav6,sales_quantity_type1,sales_quantity_type2,sales_quantity_zone1,sales_quantity_zone2,sales_quantity_zone3,sales_quantity_zone4,sales_quantity_zone5,y
0,3d8a4ae769b526187c36901f204691a663333fa4_1,1,3d8a4ae769b526187c36901f204691a663333fa4,1,-1,LG ACCESSORIES,SMALL LEATHER GOODS,COMPACT WALLETS,PF.VICTORINE,DAMIER AZUR,...,25.0,29.0,946.0,554.0,506.0,238.0,75.0,544.0,137.0,7.220374
1,c05a54f7067be054ec4b27d0d6081353ef7d9df6_1,1,c05a54f7067be054ec4b27d0d6081353ef7d9df6,0,-1,FANCY ACCESSORIES,BAG CHARMS,BAG CHARMS,BAG CHARM PETITE MALLE,SANS LIGNE ESTHETIQUE,...,39.0,NaN,338.0,60.0,84.0,55.0,25.0,155.0,79.0,6.267201
2,7cb4d3626bd48a9b523d8693266219c34aeccde8_1,1,7cb4d3626bd48a9b523d8693266219c34aeccde8,1,-1,CITY BAGS,DAILY BAGS,SHOULDER BAGS,GIROLATA,MAHINA,...,10.0,5.0,429.0,145.0,190.0,55.0,35.0,195.0,99.0,6.222576
3,8e5967f90ff9fb93aa2840fceecc537b2e8e6ff3_1,1,8e5967f90ff9fb93aa2840fceecc537b2e8e6ff3,1,-1,CITY BAGS,DAILY BAGS,SHOULDER BAGS,GIROLATA,MAHINA,...,10.0,5.0,228.0,55.0,108.0,55.0,20.0,20.0,80.0,5.624018
4,6913a128945e0efeafc52101dcdeaa610eaa4430_1,1,6913a128945e0efeafc52101dcdeaa610eaa4430,0,1,FASHION JEWELRY,NECKLACES,NECKLACES,COLL.CHARMS FOR GENTLEMEN,SANS LIGNE ESTHETIQUE,...,5.0,NaN,110.0,25.0,35.0,5.0,20.0,30.0,45.0,5.332719


### Merge Testing Dataset

In [82]:
Z = test.copy()
Z = Z.merge(navigation_stats, on = 'sku_hash', how = 'left')
Z = Z.merge(sales_stats, on = 'sku_hash', how = 'left')
Z = Z.merge(traffic_source_views, on = 'sku_hash', how = 'left')
Z = Z.merge(type_sales, on = 'sku_hash', how = 'left')
Z = Z.merge(zone_sales, on = 'sku_hash', how = 'left')

Z.loc[Z.product_type=='Accessories','product_type'] = '0'
Z.loc[Z.product_type=='Leather Goods','product_type'] = '1'
Z.product_type = Z.product_type.astype(int)

Z.loc[Z.product_gender=='Women','product_gender'] = '-1'
Z.loc[Z.product_gender=='Unisex','product_gender'] = '0'
Z.loc[Z.product_gender=='Men','product_gender'] = '1'
Z.product_gender = Z.product_gender.astype(int)

In [83]:
features = ['product_type', 'product_gender', 
            'page_views', 'sales_quantity',
            'TotalBuzzPost', 'TotalBuzz', 'NetSentiment', 'PositiveSentiment', 'NegativeSentiment', 'Impressions',
            'fr_FR_price',
            'macro_function_count', 'function_count', 'sub_function_count', 'model_count', 'aesthetic_sub_line_count', 'macro_material_count', 'color_count',
            'page_views_nav1', 'page_views_nav2', 'page_views_nav3', 'page_views_nav4', 'page_views_nav5', 'page_views_nav6',
            'sales_quantity_type1', 'sales_quantity_type2',
            'sales_quantity_zone1','sales_quantity_zone2','sales_quantity_zone3', 'sales_quantity_zone4','sales_quantity_zone5',
            'mean_target',]

### Split and Cross Validation

In [84]:


def train_test_split(tr, te, mo, feats, num_folds):

    Xtrain = []
    ytrain = []
    dtrain = []
    Xval = []
    yval = []
    dval = []

    for i in range(num_folds):
        Xtrain.append(tr.loc[(tr.month == mo) & (tr.idx != i), feats].values)
        ytrain.append(tr.loc[(tr.month == mo) & (tr.idx != i), 'y'].values)
        dtrain.append(xgb.DMatrix(Xtrain[i], ytrain[i]))

        Xval.append(tr.loc[(tr.month == mo) & (tr.idx == i), feats].values)
        yval.append(tr.loc[(tr.month == mo) & (tr.idx == i), 'y'].values)
        dval.append(xgb.DMatrix(Xval[i], yval[i]))

    Xtest = te.loc[(te.month == mo), feats].values
    dtest = xgb.DMatrix(Xtest)

    return dtrain, dval, dtest

### XGBoost Parameter

In [85]:
param = {}
param['objective'] = 'reg:linear'
param['eval_metric'] =  'rmse'
param['booster'] = 'gbtree'
param['eta'] = 0.025
param['subsample'] = 0.7
param['colsample_bytree'] = 0.7
param['num_parallel_tree'] = 3
param['min_child_weight'] = 25
param['gamma'] = 5
param['max_depth'] =  3
param['silent'] = 1


### Prediction Month 1

In [86]:
dtrain, dval, dtest = train_test_split(
    tr=X, te=Z, mo=1, feats=features, num_folds=5)

model_m1 = []
for i in range(5):
    model_m1.append(
        xgb.train(param,
                  dtrain[i],
                  50000,
                  [(dtrain[i], 'train'), (dval[i], 'eval')],
                  early_stopping_rounds=20,
                  verbose_eval=False
                  )
    )

oof_m1 = []
oof_test_m1 = []
for i in range(5):
    oof_m1.append(model_m1[i].predict(dval[i]))
    oof_test_m1.append(model_m1[i].predict(dtest))

test_m1 = np.mean(oof_test_m1, axis=0)

In [87]:
m1={}
for i in range(5):
    m1 = {**m1,**dict(zip(X.loc[(X.month==1)&(X.idx==i),'sku_hash'],oof_m1[i]))}

In [88]:
m1

{'6913a128945e0efeafc52101dcdeaa610eaa4430': 4.8146367,
 '170eb219ebb142d32d3068021552530f9895d0ef': 5.731027,
 '9911283f6be7a225917a4c544a6ea95f0e75af4c': 6.1668525,
 'c55e065bcb1d4c72bc2a9af949bb310661bb344b': 5.9474354,
 'b1234d5dd478535cb4e9966066d84ed24c0463b2': 4.989825,
 'b7d77ce215c5f55f421bbed636611831ba49bc60': 6.2630377,
 '83342f4739356582ac1ae63954b778978a317634': 4.570758,
 '59843c8d8c51d84ce1dae53dc5951ffd682b4362': 5.8049107,
 '81a12b07f6b90da9efe190b1581d553d3e3218aa': 6.3004007,
 'c7154c00150a80c84cac7a700363a0a6fbc295f9': 5.791265,
 '34e7009d03294379ae04a7c2e86772b713f10fdc': 5.4931264,
 '6c778b877ecd529be12266d4ecc2d05b81176418': 5.5587373,
 '23bf25ee278cd834c05d6a73f60517ca35507513': 5.8373876,
 '713f047a7e3798c764291733d3854100fe506da8': 5.3866982,
 'df71b29c540ed084f55764f5ca061a32847ad4f2': 6.716429,
 '384dc35dbbaaeb7cf8fa765484cad3f37e0561e8': 6.912138,
 '1b21e6b4e46346f3a635baf41beb98fa94595b79': 4.7208166,
 '554dd93733ddaa21a2a34982fff7188147180a44': 4.582873,

In [89]:
m1 = {**m1,**dict(zip(Z.loc[(Z.month==1),'sku_hash'],test_m1))}

In [90]:
m1

{'6913a128945e0efeafc52101dcdeaa610eaa4430': 4.8146367,
 '170eb219ebb142d32d3068021552530f9895d0ef': 5.731027,
 '9911283f6be7a225917a4c544a6ea95f0e75af4c': 6.1668525,
 'c55e065bcb1d4c72bc2a9af949bb310661bb344b': 5.9474354,
 'b1234d5dd478535cb4e9966066d84ed24c0463b2': 4.989825,
 'b7d77ce215c5f55f421bbed636611831ba49bc60': 6.2630377,
 '83342f4739356582ac1ae63954b778978a317634': 4.570758,
 '59843c8d8c51d84ce1dae53dc5951ffd682b4362': 5.8049107,
 '81a12b07f6b90da9efe190b1581d553d3e3218aa': 6.3004007,
 'c7154c00150a80c84cac7a700363a0a6fbc295f9': 5.791265,
 '34e7009d03294379ae04a7c2e86772b713f10fdc': 5.4931264,
 '6c778b877ecd529be12266d4ecc2d05b81176418': 5.5587373,
 '23bf25ee278cd834c05d6a73f60517ca35507513': 5.8373876,
 '713f047a7e3798c764291733d3854100fe506da8': 5.3866982,
 'df71b29c540ed084f55764f5ca061a32847ad4f2': 6.716429,
 '384dc35dbbaaeb7cf8fa765484cad3f37e0561e8': 6.912138,
 '1b21e6b4e46346f3a635baf41beb98fa94595b79': 4.7208166,
 '554dd93733ddaa21a2a34982fff7188147180a44': 4.582873,

In [91]:
oof_m1 = pd.DataFrame.from_dict(m1, orient='index').reset_index()    
oof_m1.columns = ['sku_hash', 'oof_m1']

X2 = pd.merge(X.copy(), oof_m1, on = 'sku_hash')
Z2 = pd.merge(Z.copy(), oof_m1, on = 'sku_hash')
features2 = features + ['oof_m1']  

### Prediction Month 2


In [92]:
dtrain2, dval2, dtest2 = train_test_split(tr = X2, te = Z2, mo = 2, feats = features2, num_folds = 5)

model_m2 = []

for i in range(5):
    model_m2.append(
        xgb.train(
                  param,
                  dtrain2[i],
                  50000,
                  [(dtrain2[i],'train'), (dval2[i],'eval')],
                  early_stopping_rounds = 200,
                  verbose_eval = False)
    )

# run predictions for the 2 month        
    
oof_m2 = []
oof_test_m2 = []
for i in range(5):
    oof_m2.append(model_m2[i].predict(dval2[i]))
    oof_test_m2.append(model_m2[i].predict(dtest2))
    
test_m2 = np.mean(oof_test_m2, axis=0)    
    
m2 = {}
for i in range(5):
    m2 = {**m2, **dict(zip(X.loc[(X.month==2) & (X.idx==i),'sku_hash'], oof_m2[i]))}
    
m2 = {**m2, **dict(zip(Z.loc[(Z.month==2),'sku_hash'], test_m2))}
    
oof_m2 = pd.DataFrame.from_dict(m2, orient='index').reset_index()    
oof_m2.columns = ['sku_hash', 'oof_m2']

X3 = pd.merge(X2.copy(), oof_m2, on = 'sku_hash')
Z3 = pd.merge(Z2.copy(), oof_m2, on = 'sku_hash')
features3 = features2 + ['oof_m2']    

### Prediction Month 3

In [93]:
dtrain3, dval3, dtest3 = train_test_split(tr = X, te = Z, mo = 3, feats = features, num_folds = 5)

model_m3 = []

for i in range(5):
    model_m3.append(
        xgb.train(
                  param,
                  dtrain3[i],
                  50000,
                  [(dtrain3[i],'train'),(dval3[i],'eval')],
                  early_stopping_rounds = 200,
                  verbose_eval = False)
    )

# run predictions for the 3 month        
    
oof_m3 = []
oof_test_m3 = []
for i in range(5):
    oof_m3.append(model_m3[i].predict(dval3[i]))
    oof_test_m3.append(model_m3[i].predict(dtest3))
    
test_m3 = np.mean(oof_test_m3, axis=0)    
    
m3 = {}
for i in range(5):
    m3 = {**m3, **dict(zip(X.loc[(X.month==3) & (X.idx==i),'sku_hash'], oof_m3[i]))}
    
m3 = {**m3, **dict(zip(Z.loc[(Z.month==3),'sku_hash'], test_m3))}
    
oof_m3 = pd.DataFrame.from_dict(m3, orient='index').reset_index()    
oof_m3.columns = ['sku_hash', 'oof_m3']

X3 = pd.merge(X3.copy(), oof_m3, on = 'sku_hash')
Z3 = pd.merge(Z3.copy(), oof_m3, on = 'sku_hash')

In [94]:
Z3['target'] = 0
Z3.loc[Z3.month == 1, 'target'] = Z3.loc[Z3.month == 1, 'oof_m1'] 
Z3.loc[Z3.month == 2, 'target'] = Z3.loc[Z3.month == 2, 'oof_m2'] 
Z3.loc[Z3.month == 3, 'target'] = Z3.loc[Z3.month == 3, 'oof_m3'] 

X3['pred_target'] = 0
X3.loc[X3.month == 1, 'pred_target'] = X3.loc[X3.month == 1, 'oof_m1'] 
X3.loc[X3.month == 2, 'pred_target'] = X3.loc[X3.month == 2, 'oof_m2'] 
X3.loc[X3.month == 3, 'pred_target'] = X3.loc[X3.month == 3, 'oof_m3'] 

### Evaluate Accuracy

In [95]:
print(f"month1: {np.sqrt(np.mean((X3.loc[X3.month==1,'y'] - X3.loc[X3.month==1,'pred_target'])**2))}")
print(f"month2: {np.sqrt(np.mean((X3.loc[X3.month==2,'y'] - X3.loc[X3.month==2,'pred_target'])**2))}")
print(f"month3: {np.sqrt(np.mean((X3.loc[X3.month==3,'y'] - X3.loc[X3.month==3,'pred_target'])**2))}")
print(f"overall: {np.sqrt(np.mean((X3['y'] - X3['pred_target'])**2))}")

month1: 0.4448993057058676
month2: 0.5815542552943268
month3: 0.7046403084331121
overall: 0.5867022834843385


### Product Prediction

In [96]:
Z3['target'] = np.exp(Z3.target)-1
final_sub = Z3[['ID','target']]
final_sub.to_csv(os.path.join(base_path,'silly-raddar-sub4.csv'),index=None)